In [1]:
import cards_sol_3 as cards
from ipywidgets import *

# Refactoring

Refactoring is the process of restructuring existing code to improve its readability, maintainability, and/or performance without changing its functionality.  This is something we often do implicitly as we write, but it can also be done explicitly as a separate step, particularly when we have things that may be worked on by multiple developers or created over a long period of time. When working in an agile environment, where we maintain a working but incomplete product, as we've done on this little card game, refactoring can become a critical and ongoing process, as the process of tacking on functionality to working code can yield things that are hard to maintain and extend.

## Refactoring and Debugging

Refactoring is often done at least partially in concert with debugging, as they are both things that we likely want to do as soon as we create something that works. In some cases we may find that the process of refactoring reveals bugs in our code, or makes subtle bugs easier to notice. 

Care is needed if we are both debugging and refactoring, as the goal of refactoring is to keep the functionality the same. If we are both correcting errors and changing the structure of our code, it can be hard to tell which changes are causing which effects. In most cases, it is a good idea to perform at least basic testing and debugging before refactoring, so there is a working standard to maintain as we refactor. We may well find other issues to correct as we go, but attempting to redesign, test, and fix code all at once can be extremely difficult in all but the simplest cases.

## Refactoring the Card Game

In this notebook we'll show a few steps that we can take to clean and organize our card game, along with finishing up a simple interface for playing the game. In particular, we'll make some changes to make the game's logic work more easily and intuitively in concert with the GUI that we are working on. 

One thing that I will change in the back end that makes more sense when we think about the game as a whole is to batch process the actions of each round of the game. By that I mean that players can queue up actions like submitting bets or swapping cards, and those things can be done all at once. This is a bit more realistic, as it is more like how a real game would be played, and it also holds the potential to make the logic of the game a bit more simple as we can check constraints on actions all at once, rather than having to check them as each action is taken. 

In [2]:
game = cards.FiveCardDraw()
print(game)

Player: 0:
Player 0 - : 1000
None
Player: 1:
Player 1 - : 1000
None
Player: 2:
Player 2 - : 1000
None
Player: 3:
Player 3 - : 1000
None



## Game Time

We'll make the UI for the game here. We will base it around the assumption that the game object is already created, above, and that all of our interactions - getting player info, cards, bank balances, etc. - will be done through calls to that object. 

### Layout

Since we'll now try to show more stuff on our screen, we'll create a more structured layout. We'll use a grid layout, which is more or less like a table in Word or HTML. We can define the number of rows and columns, and then place widgets in the cells of the grid. 

### Maintaining State

One thing that we need to think of when splitting the code into parts - the logic and the display - is how to maintain the state of the game. For this, I'll maintain almost everything in the game object, with only new updates being held in the GUI. Meaning that every time there's a button click that triggers a 'round' of action, that triggered action will update the game object with whatever needs to be sent in from the GUI side - such as bets and cards to swap. This is not a major change in this circumstance, but it is something to be aware of; if we have something like a bet selected here, that value isn't reflected in the logic of the game until we send that information to the game object. Every button click will resynchronize things. 

In other examples this idea also applies, likely to a greater extent. For example, if we have something that is generating aggregated data from a database, those aggregates only exist where we see them until we send them back to something permanent, like a database.

In [3]:
# Diagnostic switch
DEBUG = False

# Game constants
STARTING_BALANCE = 1000
NUM_PLAYERS = 4
CARDS = 5

# Layout constants
ROWS = 10
COLS = NUM_PLAYERS
NAME_ROW = 0
BALANCE_ROW = 6
BET_ROW = 7
WIN_ROW = 8
BUTTON_ROW = 9

# Action Functions
def submit_button_clicked(b):
    # Process swaps
    swaps = []
    swap = []
    for i, card in enumerate(toggle_buttons):
        swap.append(card.value)
        if len(swap) == 5:
            swaps.append(swap)
            swap = []
    if DEBUG:
        print(swaps)
    
    # Process bets
    bets = []
    for i, bet in enumerate(bet_sliders):
        bets.append(bet.value)
    if DEBUG:
        print(bets)
    
    # Submit data
    hands = game.submit(swaps, bets)
    if DEBUG:
        print(hands)
        print(winner)
    
    # Get results

    # Update GUI

    hands = game.getHands()
    n = 0
    for i, hand in enumerate(hands):
        for j, card in enumerate(hand):
            #print(card)
            toggle_buttons[n].description = card.__str__()
            toggle_buttons[n].value = False
            n += 1
    
    # Sliders
    for i, slider in enumerate(bet_sliders):
        slider.value = 0
    
    winner = game.calculateWinner()
    win_labels[winner].value = 'Winner!'

    new_banks = game.get_bank_balances()
    for i, bank in enumerate(new_banks):
        bank_labels[i].value = f'Bank: {bank}'

def deal_button_clicked(b):
    game.deal()

    # Update GUI
    new_banks = game.get_bank_balances()
    for i, bank in enumerate(new_banks):
        bank_labels[i].value = f'Bank: {bank}'
    
    hands = game.getHands()
    n = 0
    for i, hand in enumerate(hands):
        for j, card in enumerate(hand):
            #print(card)
            toggle_buttons[n].description = card.__str__()
            toggle_buttons[n].value = False
            n += 1
    
    for i, slider in enumerate(bet_sliders):
        slider.value = 0
    


### GUI Design

This layout of widgets is obviously a bit more elaborate than the previous one, but it is not all that different. The biggest change here is that we've made things a bit more structured and able to handle an arbitrary number of items, instead of the specific items that we need to see, like last time.

### Build UI

Create the UI elements, define the layout, add the widgets to the layout, and then display the layout.

<b>Note:</b> we could perform another round of refactoring and do something like build the UI framework here, and use a more generic update function to fill it with data. This would mean that the bits of the code below are extracted and exist only in the functions that do the refreshes. I think that it is a little more simple to do it like this - built the initial setup, then refresh it as needed. Technically though, we do have code that is redundant that we could remove. 

In [4]:
# Grid Config
grid = GridspecLayout(ROWS, COLS)
game.deal()

# Player Labels
players = game.get_player_labels()
player_labels = []
for i, player in enumerate(players):
    player_labels.append(Label(value=player))
    grid[NAME_ROW,i] = player_labels[-1]

# Toggle Buttons for Card Display
labels = game.getHands()
toggle_buttons = []
for i, player_labels in enumerate(labels):
    for j, label in enumerate(player_labels):
        toggle_buttons.append(ToggleButton(value=False, description=label, disabled=False, button_style=''))
        grid[j+1,i] = toggle_buttons[-1]

# Display Lables for Bank Balance
banks = game.get_bank_balances()
bank_labels = []
for i, bank in enumerate(banks):
    bank_labels.append(Label(value=f'Bank: {bank}'))
    grid[BALANCE_ROW, i] = bank_labels[-1]

# Slider for Bet Amount
bet_sliders = []
for i, bank in enumerate(banks):
    max = game.get_bank_balances()[i]
    bet_sliders.append(IntSlider(min=0, max=max, step=1, value=0, description='Bet:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d'))
    grid[BET_ROW,i] = bet_sliders[-1]

# Display Labels for Winning Hand
win_labels = []
for i, player in enumerate(players):
    win_labels.append(Label(value="No Win"))
    grid[WIN_ROW, i] = win_labels[-1]

# Button for Submitting Bet
submit_button = Button(description='Submit Bet', disabled=False, button_style='success', tooltip='Submit Bet')
submit_button.on_click(submit_button_clicked)
grid[BUTTON_ROW,0] = submit_button

# New Hand Button
deal_button = Button(description='New Deal', disabled=False, button_style='success', tooltip='Deal')
deal_button.on_click(deal_button_clicked)
grid[BUTTON_ROW,1] = deal_button

display(grid)

GridspecLayout(children=(Label(value='Player 0', layout=Layout(grid_area='widget001')), Label(value='Player 1'…

### Improved Code and Game

This version of the game is a little bit better in terms of functionality, but vastly superior in terms of maintainability and extensibility. I think that it is much easier to relate the GUI actions to the backend logic, add functionality, keep track of what is happening, and more. For me, the previous version felt like a real mess if I thought of the prospect of adding more functionality, this one is much more approachable.

## Function Location

One thing that we should be attentive to is where our functions and methods live. When we start to develop larger and more complex programs, particularly ones that might have vague requirements up front, we can anticipate that some of the things that we create might be in a suboptimal location. This is particularly true when we are working with a GUI, where we might have a lot of code that is related to the GUI itself, and a lot of code that is related to the underlying logic of the program.

In this case, as we've added an interface we've transitioned to having the general structure of the program to be more like:
<ul>
<li> The GUI code that lives in the notebook file. </li>
<li> The classes and their logic that live in the .py file. </li>
<li> The GUI code generally funnels its interactions with the logic of the program through the Game object, so the actions that feed the game, such as bets and card swaps, should be things that we can initiate by asking the Game object, rather than asking the individual (Player, Hand, etc...) objects directly. </li>
</ul>

So we can refactor our code to build the game class into something that can be called on to do any game related things, and then manage distributing those requests to the appropriate objects. This is a different perspective than what we've had while creating it, as we called on the individual objects to do things directly. This step will shift the design of the program to be more simple and approachable from the outside, as we are removing most of the need to care about the internal functionality of a poker game, we just need to know how to ask the game to do things. The interface that we make for our GUI to interact with our game logic should be somewhat familiar as it will be similar to the interfaces that we use each time we interact with any public library - our code can "run a poker game" and we provide a set of things we can ask that game to do which corresponds to the way that we'd use that game (not necessarily the way that the game is implemented). If we did a good job and provided good documentation, we should be able to hand our game logic to someone else, and they should be able to recreate what we've done here with any interface that they want - just as we can use all those published libraries to do things without knowing how they work internally.

For example, I've submitted the bets and card swaps all in one step to the game, rather than the original configuration where we'd do things one by one. This makes it easier to structure things in the interface similar to how the logic of the game works. This new function will then distribute all of those actions to the internal objects on its own. 

![New Function](../../images/new_method.png "New Function")
![New Function](../images/new_method.png "New Function")

<b>Note:</b> this is something that is likely not extremely obvious if you aren't actually "solving" it yourself. It is something that makes much more sense with some practice. 

## Removing Excess Code

As we work to refactor our game code, one of the main things we are doing is to create methods that will do the work in the Game class itself. This may mean that we are creating methods that supersede the functionality of methods in other classes that we've already written. In this case, we can remove the old methods from the other classes, as they are no longer needed. When we do this we are also removing potential sources of confusion or bugs, as we want to take away as many opportunities for error as we can.

An easy way to test this is to just comment out a function and run your code, if there are no errors, that function wasn't called (with consideration of inheritance and overriding). VS Code has a nice feature that will show you which functions are called and which are not, so you can see if you've missed something.

## Refactoring Tools

There are a number of tools that we can use to help us refactor our code in a more automated way. Most of these tools rely on a basic assumption that the functionality should not change, but the structure of the code can be changed. This allows them to do things like grab a function and move it to a different file, rename a variable, change arguments, or extract a block of code into a function. There are several Python refactoring packages and plugins that you can import or install in VS Code or other development environments. 

These tools are not perfect, and we can't simply write incomprehensible code and expect them to make it better at a click of a button. They can be very helpful, especially when working with a very large code base that is more likely to have some inconsistencies or redundancies. Broadly, the more organized our code is to begin with, the better results automated tools will give us - larger applications that have many classes, functions, and inheritance hierarchies are probably the most likely to benefit from these tools.

As well, while the implementation is still in its infancy, AI tools based on large language models will likely be able to grow into more capable and adaptive refactoring tools. Thier ability to capture context and meaning will allow them to make more complex changes to code. The rules of a programming language are more strictly defined than the rules of actual speech, so we can expect these tools to grow in their capability, especially when encountering code that is frequently repeated elsewhere.

This page documents the built-in refactoring tools in VS Code: https://code.visualstudio.com/docs/editor/refactoring

#### Python Notes

Some of the changes that a refactoring tool can make sound a bit magical at first glance. Python is an interpreted language, so we don't use a compiler which would translate our human-code into machine code, and potentially perform some of these steps in the process. These changes are invisible to us as programmers, but can be pretty drastic in some cases. The code that comes out the other end of a compiler will do the same thing as what went in, but it can be radically different in its structure. 

In short, tools like refactoring applications and compilers are able to examine what will happen when the code runs, and make changes that maintain that functionality but run more efficiently or have less redundancy. It isn't something that we dive into much here, but it is a good thing to be aware of to some degree. These tools can "know" what our code does, and revise to still do that thing, but in a different configuration. 

#### Gather Extension

I recently found and installed a VS Code extension called Gather, which is a simple tool that will gather all the code required to get the output of a cell in your notebook. This is useful as we can check our working code as we refactor to verify what we need to have to run each block of code. This will highlight if we are using something that has been replaced, as it can be difficult to keep track of all the things that we are using as we move things around.

## Exercise

In the example above we have some repetitive code, mainly around refreshing the UI as we go from hand to hand. Refactor the code to remove the repetition. A good strategy would be to replicate the setup stuff that we know works to draw the UI, make that into some generic "update" code, test that new thing can work on the second update (i.e. leave the original stuff), if it is good, then replace the original stuff with the new thing. A sensible end result would likely be something that can draw or redraw the GUI on demand, and we can call on it whenever we make changes without needing to manually redo those details. 

In addition, some other exercises that may help make this game awesome:
<ul>
<li> Add more rounds of betting, like a real poker game. </li>
<li> Some constraints to control what you can do when, now any item can be clicked at any time. </li>
<li> Complete the hand comparison logic to determine the winner correctly in more cases. </li>
    <ul>
    <li> Add some logic to handle ties in there as well. </li>
    </ul>
<li> Wrap a front-end shell around it - take in player names, launch a new game, etc. Like how a real video game has a menu page for setting things up. </li>
</ul>

Some of this stuff will be very particular, such as the hand values and making sure that each round of betting is processed correctly, but I don't think that any of that is drastically different or more complex, there's just more of it. If you are looking for an exercise, expanding this game is a good one. You can add or refine small bits of functionality one piece at a time, you can revise the interface without drastically changing the underlying logic, and you can test and debug as you go without too much trouble as we've made efforts to separate most of the functionality into discrete pieces.

Congratulations, you are now a professional video game developer!

In [29]:
# This may be easier to add up above near the other code. 

# Example GUI Update Functions
# Calls to these would replace the similar code in the 
# button action functions above
def updateBanks():
    banks = game.get_bank_balances()
    for i, bal in enumerate(banks):
        bank_labels[i].value = f'Bank:{bal}'
def clearSliders():
    for slider in bet_sliders:
        slider.value = 0

def updateCardsFromGame():
    hands = game.getHands()
    n = 0
    for i, hand in enumerate(hands):
        for j, card in enumerate(hand):
            #print(card)
            toggle_buttons[n].description = card.__str__()
            toggle_buttons[n].value = False
            n += 1